In [1]:
%load_ext autoreload
%autoreload 2

### Run some stuff

In [16]:
import pathlib
import numpy as np
import pandas as pd
import xarray as xr
from tqdm.notebook import tqdm
from scripts.data_readers import metric_df, model_ready_file
from model_scripts import hbv, gr4j_cema_neige


gauges = [i.stem for i in pathlib.Path("./hbv_calibrated/").glob("*.npy")]

hbv_res_df = list()
for gauge_id in tqdm(gauges):
    with xr.open_dataset(f"../geo_data/ws_related_meteo/nc_all_q/{gauge_id}.nc") as f:
        example_df = f.to_pandas()
        example_df = example_df.drop("gauge_id", axis=1)
        test_df = model_ready_file(example_df["2019-01-01":])

    with open(f"./hbv_calibrated/{gauge_id}.npy", "rb") as f:
        gauge_params = np.load(f)[0]

    res_df = pd.DataFrame()
    res_df["obs"] = test_df["Q_mm"]
    res_df["pred"] = hbv.simulation(test_df, params=gauge_params)
    hbv_res_df.append(
        metric_df(gauge_id=gauge_id, predictions=res_df["pred"], targets=res_df["obs"])
    )
hbv_res_df = pd.concat(hbv_res_df)

  0%|          | 0/1114 [00:00<?, ?it/s]

In [23]:
gauges = [i.stem for i in pathlib.Path("./gr4j_calibrated/").glob("*.npy")]

gr4j_res_df = list()
for gauge_id in tqdm(gauges):
    with xr.open_dataset(f"../geo_data/ws_related_meteo/nc_all_q/{gauge_id}.nc") as f:
        example_df = f.to_pandas()
        example_df = example_df.drop("gauge_id", axis=1)
        test_df = model_ready_file(example_df["2019-01-01":])

    with open(f"./gr4j_calibrated/{gauge_id}.npy", "rb") as f:
        gauge_params = np.load(f)[0]

    res_df = pd.DataFrame()
    res_df["obs"] = test_df["Q_mm"]
    res_df["pred"] = gr4j_cema_neige.simulation(test_df, params=gauge_params)
    gr4j_res_df.append(
        metric_df(gauge_id=gauge_id, predictions=res_df["pred"], targets=res_df["obs"])
    )
gr4j_res_df = pd.concat(gr4j_res_df)

  0%|          | 0/1105 [00:00<?, ?it/s]